In [1]:
# Let's say we want to combine the results of several different metrics 

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

import rubin_sim.maf as maf
from rubin_sim.data import get_baseline


In [3]:
baseline_file = get_baseline()
name = os.path.basename(baseline_file).replace('.db','')
out_dir = 'temp'
results_db = maf.db.ResultsDb(out_dir=out_dir)


In [5]:
class MyNestedMetric(maf.metrics.BaseMetric):
    """An example of writing a nested metirc
    """
    def __init__(self,  m5_col="fiveSigmaDepth", filter_col="filter", 
                 exptime_col="visitExposureTime", extinction_cut=0.2,
                 depth_cut=22.9, metric_name="new_nested_FoM",
                 units="rods/hogshead", badval=np.nan, **kwargs):
        # XXXprobably want to swap that depth_cut to a dict with different
        # values per filter

        maps = ["DustMap"]
        self.m5_col = m5_col
        self.filter_col = filter_col
        self.exptime_col = exptime_col

        cols = [self.m5_col, self.filter_col, self.exptime_col]
        super().__init__(cols, metric_name=metric_name, units=units, maps=maps, badval=badval, **kwargs)
        self.riz_detection_metric = maf.RIZDetectionCoaddExposureTime(ebvlim=extinction_cut)

        # setting n_filters to 0 for all these
        self.exgalm5_metric_r = maf.ExgalM5WithCuts(m5_col=m5_col, filter_col=filter_col, extinction_cut=extinction_cut,
                                                 depth_cut=depth_cut, lsst_filter="r", badval=badval, n_filters=0)
        self.exgalm5_metric_i = maf.ExgalM5WithCuts(m5_col=m5_col, filter_col=filter_col, extinction_cut=extinction_cut,
                                                 depth_cut=depth_cut, lsst_filter="i", badval=badval, n_filters=0)
        self.exgalm5_metric_z = maf.ExgalM5WithCuts(m5_col=m5_col, filter_col=filter_col, extinction_cut=extinction_cut,
                                                 depth_cut=depth_cut, lsst_filter="z", badval=badval, n_filters=0)

        # magic line so MAF knows we're returning something complicated
        self.metric_dtype = "object"

    def run(self, data_slice, slice_point):

        # set up array to hold the results
        names = ['exgal_m5, r', 'exgal_m5, i', 'exgal_m5, z', "riz_detection"]
        types = [float]*4
        result = np.zeros(1, dtype=list(zip(names, types)))
        
        result['exgal_m5, r'] = self.exgalm5_metric_r.run(data_slice, slice_point)

        result['exgal_m5, i'] = self.exgalm5_metric_i.run(data_slice, slice_point)

        result['exgal_m5, z'] = self.exgalm5_metric_z.run(data_slice, slice_point)

        result['riz_detection'] = self.riz_detection_metric.run(data_slice, slice_point)

        return result
        

In [6]:
class MyNestedSummary(maf.metrics.BaseMetric):
    """To be paired with MyNestedMetric
    """
    def run(self, data_slice, slice_point=None):
        # concatenate everything down, so we have an 
        # array with proper named columns
        data = np.concatenate(data_slice["metricdata"])

        # Do some meaningful combination of all the data
        # Note there's nothing preventing one from putting this line in
        # the main metric. Unless you have some total area from one of the
        # metrics or something, it can go above. Then probably a 
        # very simple summary metric would suffice.
        result = data["exgal_m5, r"] + 2.*data["exgal_m5, i"]/data['riz_detection']

        # Since this is a summary metric, need to reduce down to a single number
        result = np.nanmean(result)

        return result
        

In [7]:
bundle_list = []
metric = MyNestedMetric()
sql = 'note not like "%DD%" and note not like "%twi%"'
slicer = maf.slicers.HealpixSlicer(use_cache=False)
summary_metrics = [MyNestedSummary()]

bundle_list.append(maf.MetricBundle(metric, slicer, sql, run_name=name, summary_metrics=summary_metrics))



Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes


In [8]:
bd = maf.metricBundles.make_bundles_dict_from_list(bundle_list)
bg = maf.metricBundles.MetricBundleGroup(bd, baseline_file, out_dir=out_dir, results_db=results_db)
bg.run_all()

In [9]:
bundle_list[0].summary_values

{'MyNestedSummary': 26.63092802904755}